In [2]:
#非线性优化方法求解期权的隐含波动率
#隐含波动率就是使得模型估计的期权价格等于真实价格时候的sigma值
from scipy.stats import norm
import numpy as np
import scipy.optimize as so
def BSM_imply_vo(P,S,K,r,T,op_type):
    '''非线性优化方法求解期权的隐含波动率
    P:期权的市场价格；
    S:基础资产的价格；
    K:期权的执行价格；
    r:无风险收益率；
    T:期权合约的剩余期限；
    op_type:输入'call'表示看涨期权；输入'put'表示看跌期权。'''
    def f(sigma):
        d1=(np.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
        d2=d1-sigma*np.sqrt(T)
        bscall=S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
        bsput=-S*norm.cdf(-d1)+K*np.exp(-r*T)*norm.cdf(-d2)   
        if op_type=='call':
            return bscall-P #即解 bscall-p = 0 默认省略0 
        #so.fsolve函数就是一个求解非线性方程（也就是复杂方程）
#so.fsolve(function,X0)
#function代表定义的非线性方程，X0代表一个初始值（不重要，可以输入0，0.1）
#对于非线性优化方法，其解法基本都是迭代方法：给定一个起始点 X0，
#不断迭代产生新的 X1,X2,X3... ，并最终收敛至 X* 。

        else:
            return bsput-P
    return so.fsolve(f,0.2)
#这个0.2只是一个x0 不重要没有什么实际意义
        
print('非线性优化方法计算看涨期权的隐含波动率',np.round(BSM_imply_vo(0.16,5.3,6,0.03,0.5,'call'),6))
print('非线性优化方法计算看跌期权的隐含波动率',np.round(BSM_imply_vo(0.72,5.3,6,0.03,0.5,'put'),6))

非线性优化方法计算看涨期权的隐含波动率 [0.248455]
非线性优化方法计算看跌期权的隐含波动率 [0.207933]


In [3]:

#二分法求解期权的隐含波动率
def binary_imply_vo(P,S,K,r,T,op_type):
    '''二分法求解期权的隐含波动率
    P:期权的市场价格；
    S:基础资产的价格；
    K:期权的执行价格；
    r:无风险收益率；
    T:期权合约的剩余期限；
    op_type:输入'call'表示看涨期权；输入'put'表示看跌期权。'''        
    def BSM_model(S,K,sigma,r,T,op_type):
        import numpy as np
        from scipy.stats import norm
        d1=(np.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
        d2=d1-sigma*np.sqrt(T)
        if op_type=='call':
            BSM_model=S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
        else:
            BSM_model=-S*norm.cdf(-d1)+K*np.exp(-r*T)*norm.cdf(-d2) 
        return BSM_model
    sigma_down=0.000001
    sigma_up=1
    sigma_mid=(sigma_down+sigma_up)/2
    op_min=BSM_model(S,K,sigma_down,r,T,op_type)
    op_max=BSM_model(S,K,sigma_up,r,T,op_type)
    op_mid=BSM_model(S,K,sigma_mid,r,T,op_type)
    diff=P-op_mid
    if P<op_min or P>op_max:
        print('错误')
    while abs(diff)>1e-6:
#abs绝对值
#1e-6表示1乘以10的负6次方，大于等价于大于0
#我们需要差异等于0，二分法思想，当diff=0时，模型与现实相同

        #diff=P-BSM_model(S,K,sigma_mid,r,T,op_type)
        sigma_mid=(sigma_down+sigma_up)/2
        op_mid=BSM_model(S,K,sigma_mid,r,T,op_type)
        diff=P-op_mid
        if P>op_mid:
            sigma_down=sigma_mid
#当实际价格大于sigma_mid估计的op_mid时，取后半个区间继续用二分法
#如P=0.7，（0，1）为第一个区间，因为0.7大于0.5，故第二个区间为（0.5，1）
#即第二个区间的下限是第一个区间的中限
        else:
            sigma_up=sigma_mid
    return sigma_mid

print('二分法计算看涨期权的隐含波动率',np.round(binary_imply_vo(0.16,5.3,6,0.03,0.5,'call'),6))    
print('二分法计算看跌期权的隐含波动率',np.round(binary_imply_vo(0.72,5.3,6,0.03,0.5,'put'),6))

二分法计算看涨期权的隐含波动率 0.248454
二分法计算看跌期权的隐含波动率 0.207933


In [ ]:
def binary_imply_vo(P,S,K,r,T,op_type):
    def BSM_model(S,K,sigma,r,T,op_type):
        import numpy as np
        from scipy.stats import norm
        d1=(np.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
        d2=d1-sigma*np.sqrt(T)
        if op_type=='call':
            BSM_model=S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
        else:
            BSM_model=-S*norm.cdf(-d1)+K*np.exp(-r*T)*norm.cdf(-d2) 
        return BSM_model
    
    sigma_down=0.000001
    sigma_up=1
    sigma_mid=(sigma_down+sigma_up)/2
    op_min=BSM_model(S,K,sigma_down,r,T,op_type)
    op_max=BSM_model(S,K,sigma_up,r,T,op_type)
    op_mid=BSM_model(S,K,sigma_mid,r,T,op_type)
    diff=P-op_mid
    if P<op_min or P>op_max:
        print('错误')
    